In [1]:
import pandas as pd
import json
from pathlib import Path
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService

from curated_dataset_creation_helpers import create_subfolder_for_sample, open_sample_website_and_extract_html, create_empty_GDPR_files

# Load config

In [2]:
# Load config for dev dataset creation process
config_file = open(f"{str(Path.cwd())}/dev_dataset_creation_config.json")
config = json.load(config_file)
config_file.close()

# Load seeds
Load the table of seed websites. The urls will be used in the next sections of this notebook.

In [3]:
# Load tabel of all seed websites
file = pd.ExcelFile(f"{config['input_data_path']}/companies_enc.xlsx")
seeds = pd.read_excel(file, 'Companies')

# Set up Selenium

In [4]:
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.set_page_load_timeout(10)

# Inspect websites and populate dev dataset
- Set index to 0-based index of row in seeds you want to inspect and add to the dev dataset
- create a subfolder in the dev_dataset directory for that sample
- open the website and store its html inside the created subfolder
- extract GDPR contents if given

140, 105

In [8]:
seed_website_index = 105

In [9]:
create_subfolder_for_sample(config, sample_index=seed_website_index)
open_sample_website_and_extract_html(
    config,
    seeds,
    driver,
    sample_index=seed_website_index
)

[Errno 17] File exists: '/home/erik/Desktop/WiSe2022_23/IR-Project/dsgvo_handler/refactored/data/dev_dataset/105'


Now the website is opened on the browser. If there is GDPR content visible continue with next steps, if not increment index and go on with next website in the list.

In [10]:
create_empty_GDPR_files(config, sample_index=seed_website_index)

Now the files "page_source_cleaned" and "page_source_removed.html" were created in the subfolder of the currently opened website. Use browser dev tools(*) to copy the html of the GDPR element into "page_source_removed.html" and copy the websites total html without the removed GDPR element to "page_source_cleaned". After that you're done with this website. Increment the index and go on with next website in the list.

(*) How I used Google Chrome Dev tools for this:
- Open dev tools with **ctrl** + **I**
- Click the selector button in the dev tools window upper left corner (frame with curser icon)
- Click the GDPR element in the website. Now the corresponding html div is highlighted.
- Go up in the DOM tree to catch all html that wraps the GDPR content and has no other content.
- Right click "Edit as HTML" on the most outer div
- **ctrl** + **A** to select everything inside div
- **Ctrl** + **C** and past to "page_source_removed.html"
- **backspace** to remove div
- Right click "Edit as HTML" on the root html node
- **ctrl** + **A** to select everything inside html
- **Ctrl** + **C** and past to "page_source_cleaned"

# After manual dev dataset population
Run the notebook 2_automated_dataset_completion.ipynb once you're done with manually adding sample to the dev dataset. That notebook will enrich the dataset with each website's plain texts and language as well as storing the according urls.